In [2]:
import numpy as np
import cv2
from PIL import Image
import os


In [3]:

def process_images(input_folder, output_folder, process_function):
    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Get the list of subfolders within the input folder
    subfolders = next(os.walk(input_folder))[1]

    # Iterate over each subfolder
    for subfolder in subfolders:
        # Create the corresponding subfolder in the output folder
        output_subfolder = os.path.join(output_folder, subfolder)
        os.makedirs(output_subfolder, exist_ok=True)

        # Get the list of image files within the current subfolder
        subfolder_path = os.path.join(input_folder, subfolder)
        image_files = [f for f in os.listdir(subfolder_path) if os.path.isfile(os.path.join(subfolder_path, f))]

        # Process each image and save it to the output subfolder
        for image_file in image_files:
            image_path = os.path.join(subfolder_path, image_file)

            # Apply the processing function to create the new image
            processed_image = process_function(image_path)

            # Save the processed image to the output subfolder
            output_image_path = os.path.join(output_subfolder, image_file)
            processed_image.save(output_image_path)

    print("Image processing and saving completed successfully!")


In [9]:
def apply_gabor_filter(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    ksize = (9, 9)  # Kernel size
    sigma = 5.0  # Standard deviation of the Gaussian envelope
    theta = np.pi/4  # Orientation of the normal to the parallel stripes of the Gabor function
    lmbda = 10.0  # Wavelength of the sinusoidal factor
    gamma = 1.0  # Spatial aspect ratio
    psi = 0  # Phase offset
    gabor_params = cv2.getGaborKernel(ksize, sigma, theta, lmbda, gamma, psi, ktype=cv2.CV_32F)

    # Check if the image has color channels
    if len(image.shape) < 3:
        raise ValueError("Input image must have color channels")

    # Split image into color channels
    b, g, r = cv2.split(image)

    # Apply Gabor filter to each color channel
    b_filtered = cv2.filter2D(b, cv2.CV_32F, gabor_params)
    g_filtered = cv2.filter2D(g, cv2.CV_32F, gabor_params)
    r_filtered = cv2.filter2D(r, cv2.CV_32F, gabor_params)

    # Merge filtered color channels
    gabor_filtered = cv2.merge((b_filtered, g_filtered, r_filtered))

    # Normalize the filtered image for visualization
    gabor_filtered = cv2.normalize(gabor_filtered, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)

    gabor_filtered = np.array(gabor_filtered)
    gabor_filtered = Image.fromarray(gabor_filtered)

    return gabor_filtered

In [5]:
def laplacian_of_gaussian(image_path, kernel_size=7):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    blurred = cv2.GaussianBlur(gray, (kernel_size, kernel_size), 0)
    laplacian = cv2.Laplacian(blurred, cv2.CV_64F)
    laplacian = np.uint8(np.absolute(laplacian))
    laplacian = Image.fromarray(laplacian)
    return laplacian

In [6]:
input_folder = "/home/ceg98/Documents/archive/resized/resized"
output_folder = "/home/ceg98/Documents/Laplacian"

process_images(input_folder, output_folder, laplacian_of_gaussian)

Image processing and saving completed successfully!


In [10]:
input_folder = "/home/ceg98/Documents/archive/resized/resized"
output_folder = "/home/ceg98/Documents/Garbor"

process_images(input_folder, output_folder, apply_gabor_filter)

Image processing and saving completed successfully!
